In [1]:
# Ideas- 
# while saving conpressed postintgs list make
# whether to use ?|\/ for tokenization
# iterate over all elements in posting list and save them (will save lots of characters so should reduce writing time)
# To save memory - save postings list in next line and while merging only fetch terms and if they match then only get their postings list

#implemented
# could use bisect insort to insert to list in sorted order (no need now)

In [2]:
# TODO-
# save docnos to dictionary or index

# DONE- 
#dictionary list (posting lists) are not sorted currently(could do using bisect insort)
#merge multiple postings list into 1 (other file)
# make query processing (find term from dict file and then dorectly go to position in file) 
# use porter stemmer for removing words
# recheck the ignoring punchuations

In [128]:
from bs4 import BeautifulSoup
import re
import datetime
import glob
import os
import sys
import porterStemmerModule as psm
import snappy

In [136]:
collectionPath = 'tipster-ap-frac'
indexFile = 'IndexFile4'
#indexFile = 'MyTestIndex'# Change comment un comment up
stopwordFile = ''
compressionALgo = '3'
xmlTagFile = ''
tempDirectory = 'temp/'# Change 2 -> None ###IMPORTANT check / or \

In [137]:
if len(sys.argv) > 5:
    collectionPath = sys.argv[1]
    indexFile = sys.argv[2]
    stopwordFile = sys.argv[3]
    compressionALgo = sys.argv[4]
    xmlTagFile = sys.argv[5]

In [138]:
stopwordsList = []
if stopwordFile != '':
    with open(stopwordFile,'r') as f:
        lines = f.readlines()
        stopwordsList = [line.strip() for line in lines]
xmlTagsList = ['DOCNO','HEAD', 'TEXT']
if xmlTagFile != '':
    with open(xmlTagFile, 'r') as f:
        lines = f.readlines()
        xmlTagsList = [line.strip() for line in lines[1:]]

In [139]:
print(xmlTagsList)

['DOCNO', 'HEAD', 'TEXT']


In [140]:
# file = open("tipster-ap-frac/ap880212",'r')
# ap890520
def getFileContents(fileName):
    contents = []
    with open(fileName,'r') as file:
        contents = file.read()
    contents = "<start>" + contents + "</start>"
    return contents

In [127]:
def generateTermDictPart(xmlContents,docnos,term_dict = {}, xmlTagsList = ['DOCNO','HEAD', 'TEXT']):
    global stopwordsList
    ps = psm.PorterStemmer()
    docs = xmlContents.find_all('DOC')
    tokens = []
    for doc in docs:
        doc_id = doc.find_all('DOCNO')[0].get_text()
        docnos.append(doc_id)
        doc_id = len(docnos)
        #print(doc_id)
        for tag in xmlTagsList[1:]:
            tag_all = doc.find_all(tag)
            for t in tag_all:
                for localToken in filter(None, re.split("[ ,.:;\"'\n()\[\]{}`]+",t.get_text())):
                    stemmedToken = localToken.strip()
                    if not stemmedToken or stemmedToken in stopwordsList:
                        continue
                    stemmedToken = ps.stem(stemmedToken, 0, len(stemmedToken)-1)
                    stemmedToken = stemmedToken.lower()
                    #tokens.append([localToken,doc_id])
                    #print(localToken, end=" ");
                    #print("Token: ",stemmedToken)
                    if stemmedToken not in term_dict:
                        #print("Token: ", stemmedToken)
                        term_dict[stemmedToken] = [doc_id]
                    elif term_dict[stemmedToken][-1] != doc_id:
                        term_dict[stemmedToken].append(doc_id)
    return term_dict,docnos

In [64]:
def saveTermDict(term_dict, fileName):
    a = datetime.datetime.now()
    with open(tempDirectory + fileName, 'w') as file:
        for key in sorted(term_dict):
            file.write('%s %s\n'%(key, sorted(term_dict[key])))
    b = datetime.datetime.now()
    #print('save success time taken ='+ str(b-a))

In [112]:
# save term dict for set number of files together in a file
saveToFileAt = 10 # Chnage 1 to 10
fileNameIndex = 1
docnos = []
def makeDividedIndex(collectionPath, onlyFiles=False):
    print('Making Divided Index temp files')
    global fileNameIndex
    global docnos
    global xmlTagsList
    i=1
    docnos=[]
    term_dict = {}
    iterate = collectionPath
    if not onlyFiles:
        iterate = glob.glob(os.path.join(collectionPath, "*"))
    
    for fileName in iterate:
        #print(i,end=" ")
        contents = getFileContents(fileName)
        contents = BeautifulSoup(contents, 'xml')
        term_dict,docnos = generateTermDictPart(contents,docnos,term_dict, xmlTagsList)
        #print(len(docnos))
        #print("fileName", end=" ")
        if i % saveToFileAt == 0:
            saveTermDict(term_dict, "TempTermDoc-"+str(fileNameIndex))
            fileNameIndex = fileNameIndex + 1
            term_dict = {}
        i = i+1
    #save if term_dict is not empty (not a multiple of saveToFileAt)
    if bool(term_dict):
        saveTermDict(term_dict,"TempTermDoc-"+str(fileNameIndex))
        fileNameIndex = fileNameIndex + 1
    #now merge the posting list to a single file

makeDividedIndex(collectionPath) # Change uncomment and comment lower
#makeDividedIndex(['test', 'test2'], True)

Making Divided Index temp files


In [92]:
#file should be alrea by opened
def nextTerm(file):
    term = ''
    endFile = False
    postingList = []
    line = file.readline()
    if not line:
        return term, postingList, True
    #line = line[:-1].strip('][').split(', ')
    line = list(filter(None, re.split("[ ,\[\]]+",line)))
    term = line[0]
    postingList = [int(x) for x in line[1:-1]]
    #postingList = [int(x) for x in line[1:]]
    return term, postingList, endFile

In [93]:
# make it efficient (o(n)) currently(o(nlog n))
def mergeLists(lists):
    lists = [i for li in lists for i in li]
    return sorted(lists)
#     infi = 100000000000
#     lists_itr = [[i[0],0] for i in lists]
#     mergedList = []
#     lenList = sum(len(x) for x in lists)
#     itr = 0
#     while True:
#         minElement, minIndex = lists_itr[0][0], 0
#         for el, i in zip(lists_itr, range(len(lists_itr))):
#             if minElement > el[0]:
#                 minElement = el[0]
#                 minIndex = i
#         mergedList.append(minElement)
#         itr += 1
#         if itr >= lenList:
#             break
#         if len(lists[minIndex])<= lists_itr[minIndex][1]+1:
#             lists_itr[minIndex] = [infi, 0]
#         else:
#             lists_itr[minIndex] = [lists[minIndex][lists_itr[minIndex][1]+1], lists_itr[minIndex][1] + 1]
#     return mergedList

In [15]:
#mergeLists([[3], [2, 4, 5, 7, 10], [7, 9, 15, 71]])

In [16]:
# li_itr = [[i[0],0] for i in li]
# print(li_itr)
# print(li_itr[0][0])
# n = sum(len(x) for x in li)
# print(n)

In [94]:
def savePList(term, plist):
    global indexFile
    with open(indexFile + '.idx', 'ab') as file:
        file.write(bytearray(plist))

In [95]:
def savePlistNoCompression(term , plist):
    global indexFile
    with open(indexFile + '.idx', 'ab') as file:
        file.write(bytearray(plist))

In [96]:
# lst = []
# with open('this.idx', 'rb') as f:
#     f.seek(0)
#     lst = f.read(n)
#     print(lst)
#     print(bytearray(lst))
#     lst = bytearray(lst).decode()
# for i in lst:
#     print(i)
# print(list(lst))

In [97]:
def saveDict(term, postingListAt):
    global indexFile
    with open(indexFile + '.dict', 'a') as file:
        file.write('%s %s\n'%(term, postingListAt))

In [98]:
def bitstring_to_bytes(s):
    return int(s, 2).to_bytes((len(s) + 7) // 8, byteorder='big')
def bytes_to_bitstrings(li):
    ans = ''
    for i in [bin(i)[2:] for i in li]:
        if len(i)%8 != 0:
            ans = ans + '0'*(8  - len(i)%8)
        ans = ans + i
    return ans

In [99]:
def gapEncoding(postingList):
    if len(postingList) == 0:
        #print("posting List is empty for gap Encoding")
        return []
    encodedList = [postingList[0]]*len(postingList)
    for i in range(1,len(postingList)):
        encodedList[i] = postingList[i] - postingList[i-1]
    return encodedList

In [100]:
def gapDecoding(postingListEncoded):
    for i in range(1, len(postingListEncoded)):
        postingListEncoded[i] = postingListEncoded[i] + postingListEncoded[i-1]
    return postingListEncoded

In [101]:
def compressC1(postingList):
    #print("compressing C1")
    finalAns = []
    postingList = gapEncoding(postingList)
    for item in postingList:
        item_byte_array = []
        ans = ''
        binItem = bin(item)
        n = len(binItem)
        #print('item\'s binary: ',binItem)
        for i in range(n-1,1, -1):
            ans = binItem[i] + ans
            if ((n-i) % 7) == 0:
                ans = '1' + ans
        if (len(ans)%8) != 0:
            ans = "0"*(8 - (len(ans)%8)-1) + ans
            ans = '1' + ans
        ans = list(ans)
        ans[-8] = '0'
        ans = "".join(ans)
        #print("encoded binary", ans)
        for i in range(int(len(ans)/8)):
            start = i*8
            end = i*8 + 8
            #print('\tto int',ans[start:end], int(ans[start:end],2))
            item_byte_array.append(int(ans[start:end],2))
        #print('byte array for item: ', item_byte_array)
        #ans = int(ans,2)
        finalAns = finalAns + item_byte_array
        #print('in byte list:',finalAns)
    return finalAns

In [102]:
#compressC1([8019, 24711, 27689, 42921, 43210, 43402, 44079, 49625, 50737, 54429, 57261, 80205, 83298, 86796, 106040, 120846, 131611, 136697, 151642, 155925, 156058, 156103, 156104, 156291, 157516, 157627])

In [103]:
#Taking int list for compression and Returning boolean string after compression (need to convert to list of bytes)
#Taking bool string before decompression and Returning int list after compression (need to convrt list of bytes to bool str)
### IMPORTANT ----- ITS ENCODING 0 AND 1 AS 0 (SO DON'T USE 0 FOR DENOTING DOCUMENT ID OR USE +1 AND -1 FOR ALL)

class compressionC2:
    def __init__(self):
        self.postingList = ''
        self.compressedList = ''
    def l(self,x):
        return len(bin(x)[2:])
    def lsb(self,x):
        return bin(x)[3:]
    def u(self, x):
        return '1'*(x - 1) + '0'
    def compressItem(self, item):
        lx = self.l(item)
        llx = self.l(lx)
        return self.u(llx) + self.lsb(lx) + self.lsb(item)
    def compress(self, postingsList):
        postingsList = gapEncoding(postingsList)
        postingsList = [ i+2 for i in postingsList]
        compressedList = ''
        for item in postingsList:
            compressedList = compressedList + self.compressItem(item)
        return compressedList
    def decompressItem(self, compressedList):
        index = 0
        llx = 0
        lx = ''
        while compressedList[index] == '1':
            llx += 1
            index +=1
        llx+=1
        index+=1
        #print('llx:', llx, 'index: ', index)
        lx = int('1' + compressedList[index : index + llx - 1],2)
        #print('lx:', lx)
        index = index + llx - 1
        #print(compressedList[index: index + lx -1])
        x = int('1' + compressedList[index : index + lx - 1],2)
        #print('x',x)
        return x, index + lx -1
    def decompress(self, compressedList):
        postingsList = []
        index = 0
        while index < len(compressedList) and compressedList[index] == '0':
            index+=1
        while index < len(compressedList):
            item, bitsProcessed = self.decompressItem(compressedList[index : ])
            index += bitsProcessed
            postingsList.append(item)
        postingsList = [ i-2 for i in postingsList]
        postingsList = gapDecoding(postingsList)
        return postingsList

In [141]:
def compressC3(postingsList):
    if len(postingsList) == 0:
        return None
    postingsList = gapEncoding(postingsList)
    string = ''
    for el in postingsList:
        string += str(el) + ','
    return snappy.compress(string[:-1])

In [142]:
def compressPList(postingsList, algo):
    if algo == '0':
        return str(postingsList).encode()
    if algo == '1':
        return compressC1(postingsList)
    elif algo == '2':
        c2 = compressionC2()
        compList = c2.compress(postingsList)
        return bitstring_to_bytes(compList)
    elif algo == '3':
        return compressC3(postingsList)
    elif algo == '4':
        return None
    elif algo == '5':
        return None
    else:
        return None 

In [106]:
def saveEssentialsDict(stopwordsList =[], docnos=[], compressionALgo='0'):
    global indexFile
    if os.path.exists(indexFile+ '.dict'):
        os.remove(indexFile + '.dict')
    with open(indexFile+'.dict', 'a') as f:
        f.write('%s\n'%(stopwordsList))
        f.write('%s\n'%(docnos))
        f.write('%s\n'%(compressionALgo))

In [143]:
######################################## start from here
# could remove eofs and use terms = '' for breaking the loop -- leave it
import datetime
FilesWithList = fileNameIndex - 1
saveEssentialsDict(stopwordsList, docnos, compressionALgo)
def makeIndexAndDict(compressionAlgo):
    if os.path.exists(indexFile+ '.idx'):
        os.remove(indexFile + '.idx')
    terms = ['']*FilesWithList
    postingsLists = [[]]*FilesWithList
    # Path
    files = [ open(tempDirectory + 'TempTermDoc-'+ str(i+1)) for i in range(FilesWithList)]
    eofs = [False] * FilesWithList
    #initial initialisation of terms and posting list
    for i in range(FilesWithList):
        terms[i], postingsLists[i], eofs[i] = nextTerm(files[i])
    a = datetime.datetime.now()
    postingsListPointer = 0
    terms_in_dict = 0
    mergeTime =  a- a
    savingTime = a - a
    compressionTime = a - a
    while True:
        terms_in_dict += 1
        #print(terms_in_dict, end=" ")
        # find min in terms except '' (if eof is reached those terms are = ''(empty string))
        minTerm = min([term for term in terms if term != ''])
        minTermsIndex = []
        for term, index in zip(terms, range(len(terms))):
            if term == minTerm:
                minTermsIndex.append(index)
                
        #merge Postings Lists for same terms (min terms among all)
        t1 = datetime.datetime.now()
        minTermPList = mergeLists([postingsLists[i] for i in minTermsIndex])
        t2 = datetime.datetime.now()
        mergeTime = mergeTime + t2 - t1
        #print('merger :',t2-t1)
        #print('term', minTerm)
        #print('plist orig', minTermPList)
        #compress the posting lsit (got list of bytes)
        minTermPList = compressPList(minTermPList, compressionAlgo)
        #print("PList", minTermPList)
        compressionTime += datetime.datetime.now() - t2
        
        #save Postings Lists
        t3 = datetime.datetime.now()
        if compressionAlgo !='0':
            savePList(minTerm, minTermPList)
        else:
            savePlistNoCompression(minTerm, minTermPList)
        savingTime += datetime.datetime.now() - t3
        
        #save dictionary
        #print(postingsListPointer)
        saveDict(minTerm, postingsListPointer)
        #update plist pointer for dictionary
        postingsListPointer += len(minTermPList)
        
        #t3 = datetime.datetime.now()
        #print('save :', t3 - t2)
        #iterate to next terms
        for index in minTermsIndex:
            terms[index], postingsLists[index], eofs[index] = nextTerm(files[index])
        #if all eofs reached end the loop
        if all(eofs):
            break
    
    for f in files:
        f.close()
    b = datetime.datetime.now()
    print('merge time:',mergeTime)
    print('save Time:', savingTime)
    print('compressionTime:', compressionTime)
    print('Time :'+ str(b - a))
    #print(terms)
    #print(postingsLists)
makeIndexAndDict(compressionALgo)

merge time: 0:00:05.003057
save Time: 0:01:15.353521
compressionTime: 0:00:39.026950
Time :0:04:09.874291


In [285]:
# def makeCompressedIndexDict(compressionAlgo):
#     global indexFile
#     if os.path.exists(indexFile + '.dict'):
#         os.remove(indexFile + '.dict')
#     if os.path.exists(indexFile+ '.idx'):
#         os.remove(indexFile + '.idx')
#     t1 = datetime.datetime.now()
#     term = ''
#     postingsList = []
#     postingsListPointer = 0
#     with open('PostingsListNoCompression', 'r') as file: # Change 2' -> '
#         while True:
#             line = file.readline()
#             if not line:
#                 break
#             line = list(filter(None, re.split("[ ,\[\]]+",line)))
#             term = line[0]
#             postingsList = [int(x) for x in line[1:-1]]
#             postingsList = compressPList(postingsList, compressionAlgo)
#             savePList(term, postingsList)
#             saveDict(term, postingsListPointer)
#             postingsListPointer += len(postingsList)
#     print('time taken to compress postings list: ', datetime.datetime.now() - t1)

In [286]:
# t1 = datetime.datetime.now()
# makeCompressedIndexDict('2')
# print(datetime.datetime.now() - t1)

time taken to compress postings list:  0:00:00.030328
0:00:00.031065


In [258]:
# c2 = compressionC2()
# print(bin(198))

0b11000110


In [282]:
# compressed = c2.compress([1,2,3,4,5,6])
# print(compressed, len(compressed))
# print(bitstring_to_bytes(compressed))

000000 6
b'\x00'


In [283]:
# decompressed = c2.decompress(compressed)
# print(decompressed)

IndexError: string index out of range

In [255]:
# def bitstring_to_bytes(s):
#     return int(s, 2).to_bytes((len(s) + 7) // 8, byteorder='big')
# def bytes_to_bitstrings(li):
#     ans = ''
#     for i in [bin(i)[2:] for i in li]:
#         if len(i)%8 != 0:
#             ans = ans + '0'*(8  - len(i)%8)
#         ans = ans + i
#     return ans 
# # string = '1000010001000'
# # byt = bitstring_to_bytes(string)
# # print(byt)
# # print(bytes_to_bitstrings(byt))


In [234]:
# for i in b'\x02\x11':
#     print(i)

2
17


In [ ]:
# t1 = datetime.datetime.now()
# for i in range(100):
#     c2 = compressionC2([10]*10000)
# print('tiem taken: ',datetime.datetime.now() - t1)
# print(len(c2.compress()))

In [ ]:
# compressionC2([119]).compress()

In [ ]:
# def getInt(binaryNum):
#     return int(binaryNum, 2)

In [ ]:
# #assuming its bytestream stream
# def decompressC1(plist):
#     postingList = []
#     #print(len(plist))
#     numberOfBytes = int(len(plist))
#     binaryItem = ''
#     docNo = 0
#     toAddNumber = 0
#     for byt in plist:
#         #print('for',byt)
#         if byt < 128:
#             docNo = docNo*128 + byt
#             postingList.append(docNo)
#             toAddNumber = 0
#             docNo = 0
#         else:
#             byt -= 128
#             docNo = docNo*128 + byt
#             toAddNumber += 1
#     postingList = gapDecoding(postingList)
#     return postingList

In [ ]:
# 128 + 64 - 2

In [ ]:
# comp = compressC1([8019,2137991])
# print(comp)
# print(decompressC1(comp))

In [ ]:
# import sys

In [ ]:
# #testing binary file
# with open('testb','wb')as file:
#     s = 'm101'
# #     x = b'0101'
#     x = bytes(s,'ascii')
#     print(x)
#     file.write(x)

In [ ]:
# with open('testb','rb') as file:
#     x = file.read(1)
#     print(x)

In [ ]:
# with open('testb', 'wb') as file:
#     byte_arr = [1,2,3,4, 255]
#     byte_arr = bytearray(byte_arr)
#     file.write(byte_arr)
#     print(byte_arr, len(byte_arr))

In [ ]:
# with open('testb', 'rb') as file:
#     file.seek(2)
#     byte_arr = file.read(2)
#     print(byte_arr)
#     print(len(byte_arr))

In [ ]:
# bin_string = '00000011'
# byt = int(bin_string, 2)
# print(byt)

In [ ]:
# def removeStopWords(fileName):
#     file = open(fileName,'r')
#     for line in file:
#         if line in termDict:
    